# Recommendation System for Scholarship : Grant Me App



Import *Library*

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib as plt

Read the **Datasets**

In [ ]:
# Read the data from the CSV file or use an existing dataframe
data = pd.read_csv('Data_Model_V2.csv')

# Separate the columns using semicolons
data[['IPK', 'Sertifikasi', 'Sertifikasi Professional', 'Prestasi Nasional', 'Lomba/Kompetisi Top 3 Nasional',
    'Prestasi Internasional', 'Lomba/Kompetisi Top 3 Internasional', 'Intern/Magang',
    'Volunteer/Kepanitiaan']] = data['IPK;Sertifikasi;Sertifikasi Professional;Prestasi Nasional;Lomba/Kompetisi Top 3 Nasional;Prestasi Internasional;Lomba/Kompetisi Top 3 Internasional;Intern/Magang;Volunteer/Kepanitiaan'].str.split(';', expand=True)

# Drop the unnecessary columns
data = data.drop(columns=['IPK;Sertifikasi;Sertifikasi Professional;Prestasi Nasional;Lomba/Kompetisi Top 3 Nasional;Prestasi Internasional;Lomba/Kompetisi Top 3 Internasional;Intern/Magang;Volunteer/Kepanitiaan'])

# Replace empty string values with NaN
data.replace('', np.nan, inplace=True)

# Convert data types to float if needed
data[['IPK', 'Sertifikasi', 'Sertifikasi Professional', 'Prestasi Nasional', 'Lomba/Kompetisi Top 3 Nasional',
    'Prestasi Internasional', 'Lomba/Kompetisi Top 3 Internasional', 'Intern/Magang',
    'Volunteer/Kepanitiaan']] = data[['IPK', 'Sertifikasi', 'Sertifikasi Professional', 'Prestasi Nasional', 'Lomba/Kompetisi Top 3 Nasional',
    'Prestasi Internasional', 'Lomba/Kompetisi Top 3 Internasional', 'Intern/Magang',
    'Volunteer/Kepanitiaan']].astype(float)

# Replace NaN values with mean or other replacement strategy
data.fillna(data.mean(), inplace=True)

# Get the columns to be used for clustering
X = data[['IPK', 'Sertifikasi', 'Sertifikasi Professional', 'Prestasi Nasional', 'Lomba/Kompetisi Top 3 Nasional',
        'Prestasi Internasional', 'Lomba/Kompetisi Top 3 Internasional', 'Intern/Magang',
        'Volunteer/Kepanitiaan']]

Make the **Clustering Model**

In [ ]:
# Standardize the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create a k-means model with the desired number of clusters
kmeans = KMeans(n_clusters=5, random_state=42)

# Perform clustering on the data
kmeans.fit(X_scaled)

# Add the clustering result column to the dataframe
data['Cluster'] = kmeans.labels_

# Display the clustering result
print(data['Cluster'])

# Import the required libraries for model training and evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data['Cluster'], test_size=0.2, random_state=42)

# Define the TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Test the Model with **Random Input** and **Plot the Data**

In [ ]:
# Inputan baru untuk diprediksi
new_input = np.array([[3.8, 5, 3, 2, 1, 1, 1, 14, 13]])  # Ganti dengan inputan yang sesuai

# Lakukan standardisasi pada inputan baru
new_input_scaled = scaler.transform(new_input)

# Lakukan prediksi cluster
predicted_cluster = kmeans.predict(new_input_scaled)

# Tampilkan hasil prediksi
print("Inputan masuk ke cluster:", predicted_cluster)
if predicted_cluster == 0:
    print("Pemerintah")
elif predicted_cluster == 1:
    print("Swasta")
elif predicted_cluster == 2:
    print("Organisasi")
elif predicted_cluster == 3:
    print("Prestasi")
elif predicted_cluster == 4:
    print("Bantuan")
# Plotting the clusters
import matplotlib.pyplot as plt
# Plotting the clusters
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=kmeans.labels_, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker='x', color='red')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Cluster Plot')

# Define the cluster labels
cluster_labels = {
    0: "Pemerintah",
    1: "Swasta",
    2: "Organisasi",
    3: "Prestasi",
    4: "Bantuan"
}


# Display the cluster labels
for i, label in cluster_labels.items():
    plt.text(kmeans.cluster_centers_[i, 0], kmeans.cluster_centers_[i, 1], label, color='black', fontsize=12)

plt.show()

Save the **Model**

In [ ]:
# Save the model
model.save('model.h5')